In [1]:
%pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [24]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, classification_report

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [38]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target
X_test  = tfidfvect.transform(newsgroups_test.data)
y_test  = newsgroups_test.target


In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [51]:
# Calcular cuántas palabras aporta cada clase al vocabulario

class_tokens = {}
for i, doc in enumerate(newsgroups_train.data):
    # Use the vectorizer's analyzer to get tokens
    tokens = tfidfvect.build_analyzer()(doc)
    class_label = y_train[i]
    class_name = newsgroups_train.target_names[class_label]
    if class_name not in class_tokens:
        class_tokens[class_name] = set()
    class_tokens[class_name].update(tokens)

# Get the total vocabulary size
vocabulary_size = len(tfidfvect.vocabulary_)
print(f"Vocabulary size: {vocabulary_size}\n")

print("Number of unique tokens contributed by each class to the vocabulary:")
for class_name, tokens in class_tokens.items():
    num_unique_tokens = len(tokens)
    percentage = (num_unique_tokens / vocabulary_size) * 100
    print(f"{class_name}: {num_unique_tokens} ({percentage:.2f}%)")

Vocabulary size: 101631

Number of unique tokens contributed by each class to the vocabulary:
rec.autos: 8270 (8.14%)
comp.sys.mac.hardware: 7105 (6.99%)
comp.graphics: 10866 (10.69%)
sci.space: 13564 (13.35%)
talk.politics.guns: 14658 (14.42%)
sci.med: 13760 (13.54%)
comp.sys.ibm.pc.hardware: 8221 (8.09%)
comp.os.ms-windows.misc: 34683 (34.13%)
rec.motorcycles: 8462 (8.33%)
talk.religion.misc: 9145 (9.00%)
misc.forsale: 9199 (9.05%)
alt.atheism: 9021 (8.88%)
sci.electronics: 8888 (8.75%)
comp.windows.x: 11496 (11.31%)
rec.sport.hockey: 10452 (10.28%)
rec.sport.baseball: 7444 (7.32%)
soc.religion.christian: 11815 (11.63%)
talk.politics.mideast: 14378 (14.15%)
talk.politics.misc: 10710 (10.54%)
sci.crypt: 14048 (13.82%)


## Resolución del desafío

### Punto 1

Consigna: Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

Solución:
Se realiza una seleccióna aleatoria de 5 documentos del dataset de test

In [30]:
random_indices = np.random.choice(len(newsgroups_train.data), size=5, replace=False)
print(f"Índices seleccionados: {', '.join(map(str, random_indices))}")

for idx in random_indices:
  print(f"\n---Índice {idx} ---")
  #print(newsgroups_train.data[idx])
  print(f"Clase: {newsgroups_train.target_names[newsgroups_train.target[idx]]}")

Índices seleccionados: 2680, 7170, 138, 7033, 9659

---Índice 2680 ---
Clase: soc.religion.christian

---Índice 7170 ---
Clase: comp.sys.mac.hardware

---Índice 138 ---
Clase: comp.os.ms-windows.misc

---Índice 7033 ---
Clase: rec.sport.baseball

---Índice 9659 ---
Clase: talk.politics.misc


In [45]:
# contar la cantidad de ocurrencias por clase en y_train
class_counts = np.bincount(y_train)
print("Class counts in y_train:")
for class_idx, count in enumerate(class_counts):
    print(f"Class {class_idx} ({newsgroups_train.target_names[class_idx]}): {count}")

Class counts in y_train:
Class 0 (alt.atheism): 480
Class 1 (comp.graphics): 584
Class 2 (comp.os.ms-windows.misc): 591
Class 3 (comp.sys.ibm.pc.hardware): 590
Class 4 (comp.sys.mac.hardware): 578
Class 5 (comp.windows.x): 593
Class 6 (misc.forsale): 585
Class 7 (rec.autos): 594
Class 8 (rec.motorcycles): 598
Class 9 (rec.sport.baseball): 597
Class 10 (rec.sport.hockey): 600
Class 11 (sci.crypt): 595
Class 12 (sci.electronics): 591
Class 13 (sci.med): 594
Class 14 (sci.space): 593
Class 15 (soc.religion.christian): 599
Class 16 (talk.politics.guns): 546
Class 17 (talk.politics.mideast): 564
Class 18 (talk.politics.misc): 465
Class 19 (talk.religion.misc): 377


In [56]:
# obtener la cantidad de tokens que aporta cada clase al vocabulario
class_tokens = {}
vocabulary_size = len(tfidfvect.vocabulary_)
print(f"Vocabulary size: {vocabulary_size}")

for i, doc in enumerate(newsgroups_train.data):
    tokens = tfidfvect.build_analyzer()(doc) # Get tokens using the vectorizer's analyzer
    class_label = y_train[i]
    class_name = newsgroups_train.target_names[class_label]
    if class_name not in class_tokens:
        class_tokens[class_name] = set()
    class_tokens[class_name].update(tokens)


print("Number of unique tokens per class:")
for class_name, tokens in class_tokens.items():
    print(f"{class_name}: {len(tokens)} - %{100*len(tokens)/vocabulary_size:.2f}")

Vocabulary size: 101631
Number of unique tokens per class:
rec.autos: 8270 - %8.14
comp.sys.mac.hardware: 7105 - %6.99
comp.graphics: 10866 - %10.69
sci.space: 13564 - %13.35
talk.politics.guns: 14658 - %14.42
sci.med: 13760 - %13.54
comp.sys.ibm.pc.hardware: 8221 - %8.09
comp.os.ms-windows.misc: 34683 - %34.13
rec.motorcycles: 8462 - %8.33
talk.religion.misc: 9145 - %9.00
misc.forsale: 9199 - %9.05
alt.atheism: 9021 - %8.88
sci.electronics: 8888 - %8.75
comp.windows.x: 11496 - %11.31
rec.sport.hockey: 10452 - %10.28
rec.sport.baseball: 7444 - %7.32
soc.religion.christian: 11815 - %11.63
talk.politics.mideast: 14378 - %14.15
talk.politics.misc: 10710 - %10.54
sci.crypt: 14048 - %13.82


##### Documento 1

In [41]:
idx = random_indices[0]
cossim = cosine_similarity(X_train[idx], X_train)[0]
mostsim = np.argsort(cossim)[::-1][1:6]
print("Los 5 más similares son:", mostsim)

Los 5 más similares son: [ 8714  5098 11117 10995  3792]


Las 5 similitudes detectadas tienen coherencia con lo que dice el documento inicial, ya que todas tratan el mismo tema desde diferentes puntos de vista.

0) Tiene palabras como infierno, muerte y condenación eterna. El texto es un comentario de un ateo diciendo que si el infierno existe, elegiría no creer en Dios por estar en desacuerdo con la condenación eterna.
1) Tiene palabras como infierno, Jesús, Dios, biblia, Señor, cruz, eterno. Expone lo contrario al primer comentario, diciendo que el infierno existe y que Jesús predicó acerca de su existencia.
2) Tiene palabras como Cristo, infierno, Dios, vida eterna, muerte eterna. Expone un argumento similar al primero, pero desde el lado de la conciencia, es decir, si no hay conciencia eterna, no puede haber sufrimiento eterno.
3) Tiene palabras como cielo, infierno, Dios, ateos. Es comentario acerca de creer en el cielo o en el infierno, y sobre lo que creen los ateos.
4) Tiene palabras como Jesús, Dios, Cristo, pecado. Parece ser una continuación del comentario 1 o ser de la misma persona.
5) Tiene palabras como Jesús, Dios, Cinfierno, adoración. Dice que la declaración de que existe el infierno es lo que hizo que dejara de creer en Dios.

In [42]:
print("DOCUMENTO ORIGINAL:")
print("Clase:", newsgroups_train.target_names[y_train[idx]])
print("Contenido:")
print(newsgroups_train.data[idx])

print("-----------")
print("-----------")
print("Los 5 más similares son:")
for i in mostsim:
  print("Clase:", newsgroups_train.target_names[y_train[i]])
  print("Documento:")
  print(newsgroups_train.data[i])
  print("*"*50)
  print("-"*50)

DOCUMENTO ORIGINAL:
Clase: soc.religion.christian
Contenido:




True to a point.  If you were to ask a Buddhist atheist...



Actually, yes and no, Hell is eternal death.  Actually, the way 
     I've had it related to me, it's more of an eternal damnation,
     where sinners will feel the licking flames of Hell.  If I
     supposedly can feel these flames, I would assume I'm still
     alive, but suffering and away from God.


I believe Jehovah's Witnesses have a similar view, where the body
     sleeps for ever...

I don't have a problem with being condemned to Hell either.  The
     way I see it, if God wants to punish me for being honest in
     my skepticism (that is, for saying he doesn't exist), He
     certainly wouldn't be changing His nature.  Besides, I would
     rather spend an eternity in Hell than be beside God in Heaven
     knowing even one man would spend his "eternal life" being
     scorched for his wrongdoings...

Stephen

    _/_/_/_/  _/_/_/_/   _/       _/    *

##### Documento 2

Los documentos relacionados no tienen sentido con el documento propuesto. Puede ser porque la clase comp.sys.mac.hardware es la que menos palabras aporta al vovabulario.

0) Tiene palabras como consola, monitor y también precios. Una persona pide recomentaciones acerca de un Power Center para proteger su computadora de un aumento de tensión. Insinúa que quiere comprar uno y nombra las marcas y precio que estuvo viendo
1) Es un vendedor de CDs. Muestra las marcas y precios.
2) Es un relato sobre una masacre al pueblo armenio. No tiene que ver con el primer documento. La única similitud posible que encuentro es que tiene números.
3) El documento habla de dispositivos electrónicos y de proteger las conexiones de cortocircuitos y sobrecalentamiento. No es el mismo tema, pero es bastante más parecido que los demás.
4) Es una reunión pública en la casa blanca. No tiene que ver con el primer documento. La única similitud posible que encuentro es que tiene números.
5) Es un counicado de prensa de la casa blanca. No tiene que ver con el primer documento. La única similitud posible que encuentro es que tiene números.

In [44]:
idx = random_indices[1]
cossim = cosine_similarity(X_train[idx], X_train)[0]
mostsim = np.argsort(cossim)[::-1][1:6]
print("Los 5 más similares son:", mostsim)

print("DOCUMENTO ORIGINAL:")
print("Clase:", newsgroups_train.target_names[y_train[idx]])
print("Contenido:")
print(newsgroups_train.data[idx])

print("-----------")
print("-----------")
print("Los 5 más similares son:")
for i in mostsim:
  print("Clase:", newsgroups_train.target_names[y_train[i]])
  print("Documento:")
  print(newsgroups_train.data[i])
  print("*"*50)
  print("-"*50)

Los 5 más similares son: [5764 9623 4977 4271 6894]
DOCUMENTO ORIGINAL:
Clase: comp.sys.mac.hardware
Contenido:
Hi There,
	I am trying to find out a reliable Power Center, it is basically
a surge protector that sits below the monitor and has individual control
for each outlet. Some people have an opinion that none of them work well.
The ones that I could locate in Microcenter catalogue were:

Tripp Lite's Isobar Command COnsole ($79)

Proxima Power Director (89.95)

Kensington Masterpiece Plus (109.95)

Has anyone used one of these? Could you please send me your feedback
on these?

thanks
-----------
-----------
Los 5 más similares son:
Clase: misc.forsale
Documento:
Hi!

I have the following 2 CD's for sale. These are absolutely new and in
the original packing.

Artist		Album				Original	Sale
						 Price		 Price

Madonna		The Immaculate Collection 	$19.95+		$11.95

Pet Shop Boys	Discography			$19.95		$11.95

If you are interested, pls. contact me at:

			parikhma@ucunix.san.uc.edu

T

##### Documento 3

Los documentos relacionados tienen sentido con el documento propuesto. Aquí lo más relevante fue la firma del usario que envió el mail, pero aún así pudo identificar emails que responden a los temas que habla; es decir, relaciona el tema pedido.

0. Es un email de una persona cuyo nombre/firma aparece en el remitente. Está buscando un archivo porque cambió de dirección.

1., 2., 3. Son emails de la misma persona donde esta hace una consulta o responde.

4. Parece una respuesta al email 3 sobre el uso de caracteres.
5. Parece la respuesta al email 0 donde se indica un directorio para buscar un archivo.

In [57]:
idx = random_indices[2]
cossim = cosine_similarity(X_train[idx], X_train)[0]
mostsim = np.argsort(cossim)[::-1][1:6]
print("Los 5 más similares son:", mostsim)

print("DOCUMENTO ORIGINAL:")
print("Clase:", newsgroups_train.target_names[y_train[idx]])
print("Contenido:")
print(newsgroups_train.data[idx])

print("-----------")
print("-----------")
print("Los 5 más similares son:")
for i in mostsim:
  print("Clase:", newsgroups_train.target_names[y_train[i]])
  print("Documento:")
  print(newsgroups_train.data[i])
  print("*"*50)
  print("-"*50)

Los 5 más similares son: [6494 8040 6453 8074 2341]
DOCUMENTO ORIGINAL:
Clase: comp.os.ms-windows.misc
Contenido:
Ok, then where is the info for the Licensing kept?  Which file?  In the
organization box I put my address, and when I moved, I wanted to change it, but
couldn't find it.  I could find my name, but not the organization.

  ------------------------------------------------------------------------
  | Robert S. Dubinski |  Aliases include:  Robb, Regal, Sir, Mr., and I |
  ------------------------------------------------------------------------
  | Marquette University ||||||||||| Math / Computer Science Double-Major|
  ------------------------------------------------------------------------
  | Internet Address: 2A42Dubinski.vms.csd.mu.edu  |	Milwaukee, WI    |
-----------
-----------
Los 5 más similares son:
Clase: comp.os.ms-windows.misc
Documento:
	Thanks, I think I've figured it out now.

  ------------------------------------------------------------------------
  | Robert

##### Documento 4

La clase es la segunda que menos token aporta. Al parecer pesaron más las palabra de conservadurismo que la descripción y comparación de jugadores. El único documento que tiene una realción fuerte con el original es el número 3.

0. Habla de baseball, compara varios jugadores con sus habilidades y debilidades y de estrategias. Habla de que la estrategia de un equipo es conservadora.

1. Vuelve a aparecer la historia de la masacre armenia que apareció previamente. Es un texto muy largo, quizás por la cantidad de palabras que provee el documento, se lo relacione con algunos documentos a los que les cuesta encontrar relación. También podemos decir que el uso de la palabra conservador puede generar esta relación.
2. Es un consejo para alguien agnóstico cuyo amigo es cristiano y quiere convencerlo de su punto de vista. Llama a algunos cristianos "fundamentacistas". Imagino que la palabra consevador tuvo que ver con la relación que se armó con este documento.
3. Trata los mimos temas que el documento 0.
4. Otro relato sobre la profanación de los derechos humanos de armenios
5. Es un argumento acerca de que la persona no cree en Dios porque es como un tirano.

In [59]:
idx = random_indices[3]
cossim = cosine_similarity(X_train[idx], X_train)[0]
mostsim = np.argsort(cossim)[::-1][1:6]
print("Los 5 más similares son:", mostsim)

print("DOCUMENTO ORIGINAL:")
print("Clase:", newsgroups_train.target_names[y_train[idx]])
print("Contenido:")
print(newsgroups_train.data[idx])

print("-----------")
print("-----------")
print("Los 5 más similares son:")
for i in mostsim:
  print("Clase:", newsgroups_train.target_names[y_train[i]])
  print("Documento:")
  print(newsgroups_train.data[i])
  print("*"*50)
  print("-"*50)

Los 5 más similares son: [9623 9670 3484 1292 5826]
DOCUMENTO ORIGINAL:
Clase: rec.sport.baseball
Contenido:

  The same Bill James? Why do you say that? It sounds like you're suggesting 
Bill James had something to do with overhyping the kid to death. Au contraire;
he was fairly critical of him after his ROY campaign, noting that he wasn't
all-world as a catcher or a hitter. He called him basically average when
everyone *else* in the media was predicting the next Johnny Bench or Roy
Campanella. 


  Which reports are those? 


  I like Hernandez a lot, but if Piazza can catch the ball, you've gotta play
him IMHO. He's a much better hitter, although Hernandez isn't a *bad* hitter.
Right now, it sounds like Piazza will catch most of the time and Hernandez
will be Candiotti's caddy since he can catch the knuckler. As long as they
play up to their abilities, the Dodgers could have a very good catching
tandem.  


  I think both are overrated defensively (see Nichols' Law of catcher
defens

##### Documento 5

El texto original parece ser lo suficientemente largo y estar estructurado de una forma muy similar en cuanto a temas hablados y formato de las rondas de prensa. También se identifica uno de los temas tratados y se relaciona con eventos dedidados a dicho tema.

0. Es una ronda de prensa en la casa blanca. Hablan acerca de temas variados: asuntos diplomáticos, empleos de verano para jóvenes y asuntos sociales.
1. Es otra ronda de prensa en la casa blanca. Se hablan temas similares (asuntos diplomáticos, empleos y asuntos sociales) aunque no iguales.
2. Ronda de prensa en la casa blanca. Hablan acerca de la agenda del presidente, asuntos diplomáticos, empleos y política exterior.
3. Es un evento sobre uno de los temas que se tratan el documento original (empleos de verano para jóvenes)
4. Es un evento sobre uno de los temas que se tratan el documento original (empleos de verano para jóvenes)
5. Discurso del presidente sobre el tema de empleos para jóvenes y otros temas sociales.

In [60]:
idx = random_indices[4]
cossim = cosine_similarity(X_train[idx], X_train)[0]
mostsim = np.argsort(cossim)[::-1][1:6]
print("Los 5 más similares son:", mostsim)

print("DOCUMENTO ORIGINAL:")
print("Clase:", newsgroups_train.target_names[y_train[idx]])
print("Contenido:")
print(newsgroups_train.data[idx])

print("-----------")
print("-----------")
print("Los 5 más similares son:")
for i in mostsim:
  print("Clase:", newsgroups_train.target_names[y_train[i]])
  print("Documento:")
  print(newsgroups_train.data[i])
  print("*"*50)
  print("-"*50)

Los 5 más similares son: [9120 6933 4271 6894 6635]
DOCUMENTO ORIGINAL:
Clase: talk.politics.misc
Contenido:
THE WHITE HOUSE

                    Office of the Press Secretary
_____________________________________________________________________
For Immediate Release                                  April 14, 1993



                            PRESS BRIEFING
                       BY GEORGE STEPHANOPOULOS


                          The Briefing Room


12:40 P.M. EDT
	     
	     
	     MR. STEPHANOPOULOS:  I guess I'm just prepared to take 
questions today.
	     
	     Q	  George, Bob Dole says that the Clinton 
administration's policy on Bosnia is a failure and that he wants the 
United States to take the lead in lifting the arms embargo so that 
the Bosnian Muslims can defend themselves.
	     
	     MR. STEPHANOPOULOS:  As you know, President Clinton has 
said that that suggestion is under active consideration.  Obviously, 
this is a tragic situation in Bosnia.  And if the Bosnia

### Punto 2

Consigna:
Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

In [22]:
y_pred_zero_shot = []
for i, test_doc_vector in enumerate(X_test):
    # Calculo de la similitud por coseno con los ducumentos de train
    cossim = cosine_similarity(test_doc_vector, X_train)[0]

    # Obtengo el índice más similar
    most_similar_idx = np.argmax(cossim)

    # Asigno la clase del documento más similar
    predicted_class = y_train[most_similar_idx]
    y_pred_zero_shot.append(predicted_class)

y_pred_zero_shot = np.array(y_pred_zero_shot)

In [42]:
# Evaluación del modelo
f1_macro_zero_shot = f1_score(y_test, y_pred_zero_shot, average='macro')
accuracy = accuracy_score(y_test, y_pred_zero_shot)
print(f"F1-score (macro): {f1_macro_zero_shot}")
print("Accuracy:",accuracy )
print(classification_report(y_test, y_pred_zero_shot, target_names=newsgroups_train.target_names, digits=3))

F1-score (macro): 0.5049911553681621
Accuracy: 0.5088953797132236
                          precision    recall  f1-score   support

             alt.atheism      0.366     0.508     0.425       319
           comp.graphics      0.543     0.483     0.512       389
 comp.os.ms-windows.misc      0.506     0.457     0.480       394
comp.sys.ibm.pc.hardware      0.515     0.520     0.518       392
   comp.sys.mac.hardware      0.535     0.499     0.516       385
          comp.windows.x      0.701     0.592     0.642       395
            misc.forsale      0.629     0.462     0.533       390
               rec.autos      0.406     0.576     0.476       396
         rec.motorcycles      0.635     0.515     0.569       398
      rec.sport.baseball      0.645     0.537     0.586       397
        rec.sport.hockey      0.748     0.722     0.735       399
               sci.crypt      0.552     0.588     0.570       396
         sci.electronics      0.531     0.328     0.406       393
         

Conclusión: zero-shot no parece ser una buena opción en general. Sin embargo, al mirar las métricas en cada clase, hay algunas con bastante eficiencia para ser un algoritmo tan simple.
Los grupos `rec.sport.hockey` y `comp.windows.x` tuvieron mejores métricas, locual podría deberse a que estas clases contienen vocabulario más distintivo o patrones de lenguaje que resultan en una mayor similaridad y palabras que no suelen ser frecuentes en las demás clases.

Un F1-score macro de alrededor del 50% indica que el modelo tiene un rendimiento limitado en la mayoría de las clases.

El enfoque zero-shot puede ser útil en escenarios donde no se dispone de datos de entrenamiento etiquetados para todas las clases.

### Punto 3

Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

NO cambiar el hiperparámetro ngram_range de los vectorizadores.

Solución:

A continuación se realiza una búsqueda de hiperparámetros simple buscando optimizar f1_macro en ambos entrenamientos (MultinomialNB y ComplementNB) usando cross validation

In [43]:
from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

#HP para el vectorizador
vectorizer_params = {
    'tfidfvect__use_idf': (True, False),
    'tfidfvect__norm': ('l1', 'l2'),
    'tfidfvect__max_df': (0.9, 0.95, 1.0),
    'tfidfvect__min_df': (1, 2, 3),
}

multinomial_nb_params = {
    'clf__alpha': (0.1, 0.5, 1.0),
}

complement_nb_params = {
    'clf__alpha': (0.1, 0.5, 1.0),
}

# Pipelines manteniendo ngram_range=(1,1)
pipeline_mnb = Pipeline([('tfidfvect', TfidfVectorizer(ngram_range=(1, 1))), ('clf', MultinomialNB())])
pipeline_cnb = Pipeline([('tfidfvect', TfidfVectorizer(ngram_range=(1, 1))), ('clf', ComplementNB())])

#### Modelo: Naïve Bayes Multinomial

In [45]:

print("Buscando hiperparámetros...")
grid_search_mnb = GridSearchCV(pipeline_mnb, {**vectorizer_params, **multinomial_nb_params}, cv=3, scoring='f1_macro', n_jobs=-1)
grid_search_mnb.fit(newsgroups_train.data, y_train)

print("Mejores hiperparámetros:")
print(grid_search_mnb.best_params_)
print(f"Mejor métrica F1-score macro: {grid_search_mnb.best_score_}")


Buscando hiperparámetros...
Mejores hiperparámetros:
{'clf__alpha': 0.1, 'tfidfvect__max_df': 0.9, 'tfidfvect__min_df': 2, 'tfidfvect__norm': 'l2', 'tfidfvect__use_idf': True}
Mejor métrica F1-score macro: 0.7246222432600397


In [46]:
best_mnb_model = grid_search_mnb.best_estimator_
y_pred = best_mnb_model.predict(newsgroups_test.data)

# Evaluación del modelo
f1_macro = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)
print(f"F1-score (macro): {f1_macro}")
print("Accuracy:",accuracy )
print(classification_report(y_test, y_pred, target_names=newsgroups_train.target_names, digits=3))

F1-score (macro): 0.6726552851196274
Accuracy: 0.69676048858205
                          precision    recall  f1-score   support

             alt.atheism      0.650     0.367     0.469       319
           comp.graphics      0.645     0.715     0.678       389
 comp.os.ms-windows.misc      0.678     0.551     0.608       394
comp.sys.ibm.pc.hardware      0.614     0.722     0.664       392
   comp.sys.mac.hardware      0.747     0.696     0.720       385
          comp.windows.x      0.826     0.747     0.785       395
            misc.forsale      0.841     0.744     0.789       390
               rec.autos      0.787     0.747     0.767       396
         rec.motorcycles      0.803     0.759     0.780       398
      rec.sport.baseball      0.921     0.819     0.867       397
        rec.sport.hockey      0.602     0.925     0.729       399
               sci.crypt      0.703     0.773     0.736       396
         sci.electronics      0.717     0.567     0.634       393
           

#### Modelo: ComplementNB

In [48]:

print("Buscando hiperparámetros...")
grid_search_cnb = GridSearchCV(pipeline_cnb, {**vectorizer_params, **complement_nb_params}, cv=3, scoring='f1_macro', n_jobs=-1)
grid_search_cnb.fit(newsgroups_train.data, y_train)

print("Mejores hiperparámetros:")
print(grid_search_cnb.best_params_)
print(f"Mejor métrica F1-score macro: {grid_search_cnb.best_score_}")


Buscando hiperparámetros...
Mejores hiperparámetros:
{'clf__alpha': 0.1, 'tfidfvect__max_df': 0.9, 'tfidfvect__min_df': 1, 'tfidfvect__norm': 'l2', 'tfidfvect__use_idf': False}
Mejor métrica F1-score macro: 0.7551256958290183


In [49]:
best_cnb_model = grid_search_cnb.best_estimator_
y_pred = best_cnb_model.predict(newsgroups_test.data)

# Evaluación del modelo
f1_macro = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)
print(f"F1-score (macro): {f1_macro}")
print("Accuracy:",accuracy )
print(classification_report(y_test, y_pred, target_names=newsgroups_train.target_names, digits=3))

F1-score (macro): 0.6987411616066841
Accuracy: 0.7184014869888475
                          precision    recall  f1-score   support

             alt.atheism      0.323     0.455     0.378       319
           comp.graphics      0.719     0.725     0.722       389
 comp.os.ms-windows.misc      0.703     0.584     0.638       394
comp.sys.ibm.pc.hardware      0.650     0.681     0.665       392
   comp.sys.mac.hardware      0.798     0.727     0.761       385
          comp.windows.x      0.811     0.782     0.796       395
            misc.forsale      0.717     0.787     0.751       390
               rec.autos      0.813     0.735     0.772       396
         rec.motorcycles      0.832     0.786     0.809       398
      rec.sport.baseball      0.910     0.844     0.876       397
        rec.sport.hockey      0.869     0.945     0.905       399
               sci.crypt      0.794     0.801     0.797       396
         sci.electronics      0.712     0.565     0.630       393
         

Conclusión:

Ambos modelos mejoraron la performance respecto del zero-shot. Esto sugiere que un enfoque de clasificación supervisada utilizando características de texto (TF-IDF) es mucho más efectivo para esta tarea.

Si miramos las métricas de cada clase, se observa una mejora constante en las métricas de cada clase.

Las clases con un mayor número de palabras aportadas al vocabulario tienden a tener mejores métricas de rendimiento, lo cual es esperable ya que tienen más información distintiva para la clasificación.

En resumen, ambos modelos mejores opciones para clasificación de texto, con ComplementNB mostrando una ligera ventaja en el F1-score macro general y en particular en la mayoría de las clases.

### Punto 4

Consigna: Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.

In [78]:
# Transponer la matriz documento-término para obtener una matriz término-documento
X_train_transposed = X_train.T

# Defino las 5 parabras a analizar:
words_to_analyze = ['women', 'rights', 'religion', 'moon', 'space']

for word in words_to_analyze:
      # Obtengo el índice de la palabra i en el vocabulario
      word_idx = tfidfvect.vocabulary_[word]

      # Obtengo el vector asociado en la matriz traspuesta
      word_vector = X_train_transposed[word_idx]

      # Similitud por coseno
      cossim_words = cosine_similarity(word_vector, X_train_transposed)[0]

      # Obtengo los índices de las palabras más similares
      most_similar_word_indices = np.argsort(cossim_words)[::-1][1:6]

      print(f"\n--- Palabras similares a '{word}' ---")
      for i in most_similar_word_indices:
          print(f"- {idx2word[i]}")


--- Palabras similares a 'women' ---
- sexist
- maternity
- widow
- brainsex
- lucio

--- Palabras similares a 'rights' ---
- grantor
- civil
- amerikan
- entitiled
- anticipate

--- Palabras similares a 'religion' ---
- religious
- religions
- categorized
- purpsoe
- crusades

--- Palabras similares a 'moon' ---
- lunar
- phases
- gravitacional
- atraction
- sattellite

--- Palabras similares a 'space' ---
- nasa
- seds
- shuttle
- enfant
- seti


### Conclusiones sobre la similaridad entre palabras

- **women**: Las palabras más similares a 'women' parecen estar relacionadas con discusiones sobre género, roles sociales y temas potencialmente controversiales (`sexist`). `maternity` y `widow` se refieren a estados o roles asociados a las mujeres. `brainsex` y `lucio` podrían ser términos específicos de alguna discusión particular dentro del dataset.

- **rights**: Las palabras similares a 'rights' se relacionan fuertemente con conceptos legales y sociales, como era de esperarse. `civil rights` es una asociación directa. `grantor` y `entitiled` tienen connotaciones legales. `amerikan` podría referirse a discusiones sobre derechos en el contexto estadounidense. `anticipate` es menos directa, pero puede estar relacionada a discusiones sobre anticipar derechos.

- **religion**: Las palabras similares a 'religion' están muy relacionadas con el tema religioso en sí. `religious` y `religions` son variaciones obvias. `crusades` es un evento histórico relacionado con la religión. `categorized` y `purpsoe` (posiblemente un error tipográfico de 'purpose') podrían estar relacionadas con discusiones sobre cómo se clasifican las religiones o su propósito.

- **moon**: Las palabras similares a 'moon' están claramente vinculadas a la astronomía y conceptos espaciales. `lunar` y `phases` se refieren directamente a características de la luna. `gravitacional`, `atraction` y `sattellite` son conceptos físicos y objetos relacionados con cuerpos celestes.

- **space**: Las palabras similares a 'space' están fuertemente asociadas con la exploración espacial y organizaciones relacionadas. `nasa` y `seds` son organizaciones espaciales. `shuttle` se refiere a un vehículo espacial. `seti` está relacionado con la búsqueda de inteligencia extraterrestre. `enfant` parece ser una palabra menos directamente relacionada y podría ser ruido o depender de un contexto muy específico dentro de los documentos.

En general, la similaridad de palabras basada en la transposición de la matriz TF-IDF parece capturar relaciones semánticas y contextuales relevantes. Las palabras más similares a los términos consultados a menudo pertenecen al mismo dominio temático, lo que valida el enfoque de vectorización de palabras de esta manera. Sin embargo, algunas asociaciones pueden ser menos obvias y requerir una inspección manual de los documentos para comprender completamente la conexión.